In [15]:
import google.generativeai as genai
import pandas as pd
import time
import numpy as np
from scipy import stats
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [16]:
#reset annotazioni
'''
annotated_df = pd.read_csv("genai/v2/fewshot_polarization_3classes_classifications.csv", sep=",", encoding="utf-8")
annotated_df.to_csv("data/annotated_dataset/polarization_annotated_data.csv", sep=",", encoding="utf-8", index=False)
'''

'\nannotated_df = pd.read_csv("genai/v2/fewshot_polarization_3classes_classifications.csv", sep=",", encoding="utf-8")\nannotated_df.to_csv("data/annotated_dataset/polarization_annotated_data.csv", sep=",", encoding="utf-8", index=False)\n'

### GEN AI

In [ ]:
#API_KEY = ""
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

In [23]:
def classify_record(text_id, text, prompt, model):
    filled_prompt = prompt.replace('[$$$]',text)
    res = model.generate_content(filled_prompt)
    time.sleep(4)
    return str(text_id + "," +res.text)

In [24]:
#CALCOLO CLASSIFICAZIONI GEMINI
def calcola_classificazioni(prompt):
    cleaned_df = pd.read_csv("data/cleaned_dataset/merged.tsv", sep=",", encoding="utf-8")
    annotated_df = pd.read_csv("data/annotated_dataset/polarization_annotated_data.csv", sep=",", encoding="utf-8")
    df_to_annotate = cleaned_df[~cleaned_df['ID'].isin(annotated_df['id'])]

    #Estraggo i testi da dare in input a gemini
    texts_to_classify = []
    for id in df_to_annotate["ID"]:
        texts_to_classify.append((id, df_to_annotate[df_to_annotate["ID"] == id].text.iloc[0]))

    i = 0
    gemini_classes = []
    for tupla in texts_to_classify:
        res = classify_record(tupla[0], tupla[1], prompt, model)
        gemini_classes.append(res)
        i+=1

        if i%5 == 0:
            print(i, end = " - ")

            classification_list = []
            for stringa in gemini_classes:
                stringa = stringa.strip("\n")
                lista = stringa.split(",")
                lista = [lista[0]] + [int(x) for x in lista[1:]]
                classification_list.append(lista)

            gemini_classes = []
            classification_df = pd.DataFrame(classification_list, columns = ['id', 'polarization'])
            annotated_df = pd.read_csv("data/annotated_dataset/polarization_annotated_data.csv", sep=",", encoding="utf-8")
            pd.concat([annotated_df, classification_df], ignore_index=True).to_csv("data/annotated_dataset/polarization_annotated_data.csv", index=False)

In [25]:
with open('genai/v2/fewshot_3classes_polarization_prompt.txt', 'r', encoding="utf8") as f:
    prompt = f.read()

In [26]:
calcola_classificazioni(prompt)

5 - 10 - 15 - 20 - 25 - 30 - 35 - 40 - 45 - 50 - 55 - 60 - 65 - 70 - 75 - 80 - 85 - 90 - 95 - 100 - 105 - 110 - 115 - 120 - 125 - 130 - 135 - 140 - 145 - 150 - 155 - 160 - 165 - 170 - 175 - 180 - 185 - 190 - 195 - 200 - 205 - 210 - 215 - 220 - 225 - 230 - 235 - 240 - 245 - 250 - 255 - 260 - 265 - 270 - 275 - 280 - 285 - 290 - 